In [41]:
# -1-) Notlar
"""
    Her ilce icin esit sayida veri yok. Bu sayilar ilce_tarih_sayilari degiskeninde tutulu.

    Yapilmasi gerekenler:
        - Weather'daki verileri gunluge cevir. ++++
        - Her ilce icin hava durumu verilerini(bir gun icin ortalama alaraktan), tarihleri, tatil_flag'lari
        kesinti sayilarini tutan bir DataFrame olustur. ++++
        - Kesinti sayilarinin tarihe bagli ayri ayri grafiklerini cikart. (veya beraber de olabilir)
        - Hava kosullarindan iyi, orta, kotu, cok kotu gibi bir bilgi cikartmaya calis. Belki burada yapay zeka
        kullanabilirsin. orda bir formül belirlemek lazim ona göre siniflandirilir.
        
"""

"\n    Her ilce icin esit sayida veri yok. Bu sayilar ilce_tarih_sayilari degiskeninde tutulu.\n\n    Yapilmasi gerekenler:\n        - Weather'daki verileri gunluge cevir.\n        - Her ilce icin hava durumu verilerini(bir gun icin ortalama alaraktan), tarihleri, tatil_flag'lari\n        kesinti sayilarini tutan bir DataFrame olustur.\n        - Kesinti sayilarinin tarihe bagli ayri ayri grafiklerini cikart. (veya beraber de olabilir)\n        - Hava kosullarindan iyi, orta, kotu, cok kotu gibi bir bilgi cikartmaya calis. Belki burada yapay zeka\n        kullanabilirsin.\n        \n"

In [62]:
# 0-) Import required moduls and libraries

import pandas as pd
import numpy as np
from datetime import datetime, timezone, timedelta
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import math
import os
from unidecode import unidecode # to convert Turkish characters to English
# bildirimisiz_sum tahmin edilecek

In [65]:
# 1-) read and preproccess train.csv
train = pd.read_csv("./train.csv") # 46.944 satir, 4 kolon

print(train) # 1.098 farkli tarih var, 47 farkli ilce var

tarihler = []
for i in train["tarih"]:
    tarihler.append(datetime.strptime(i, "%Y-%m-%d"))
train["tarih"] = tarihler

print(train.dtypes)

dict = {}
for label, group in train.groupby("ilce"):
    dict[label] = group

ilceler = (list(dict.keys()))
print(dict) # keys olarak her ilceyi, values olarak o ilcelerin bulundugu satirlari icerir

            tarih            ilce  bildirimsiz_sum  bildirimli_sum
0      2021-01-01  manisa-akhisar                9               0
1      2021-01-02  manisa-akhisar               10               0
2      2021-01-03  manisa-akhisar                5               2
3      2021-01-04  manisa-akhisar                6               0
4      2021-01-05  manisa-akhisar               15               4
...           ...             ...              ...             ...
46939  2023-12-28    izmir-selcuk                1               0
46940  2023-12-29    izmir-selcuk                1               1
46941  2023-12-30    izmir-selcuk                0               2
46942  2023-12-31    izmir-selcuk                1               0
46943  2024-01-03    izmir-selcuk                3               0

[46944 rows x 4 columns]
tarih              datetime64[ns]
ilce                       object
bildirimsiz_sum             int64
bildirimli_sum              int64
dtype: object
{'izmir-aliaga':    

In [66]:
# 2-) extract ilce and keep preprocessing train.csv
"""
for label in dict.keys(): # her ilce icin bildirimsiz ve bildirimli olarak grafiklerini cikart
    print(dict[label]["bildirimsiz_sum"])
    plt.figure(figsize=(14,6))
    plt.bar(dict[label]["tarih"],dict[label]["bildirimsiz_sum"])
    plt.title(label)
    plt.xticks(rotation=90)
    plt.show()
"""
"""
plt.figure(figsize=(14,6))
plt.bar(dict["izmir-konak"]["tarih"],dict["izmir-konak"]["bildirimsiz_sum"])
plt.title(label)
plt.margins(0.01)
plt.xticks(rotation=90)
plt.show()
"""
# ilce tarih sayilarini al hepsinde esit veri yok
ilce_tarih_sayilari = {}
for name in dict.keys():
    ilce_tarih_sayilari[name] = len(list(dict[name]["tarih"].to_dict().values()))

print(ilce_tarih_sayilari)
for name in dict.keys():
    dict[name].set_index("tarih", inplace=True)

train.set_index("tarih", inplace=True) # train'in tarih kolonunu indexe cevir
print(train)

{'izmir-aliaga': 1078, 'izmir-balcova': 681, 'izmir-bayindir': 1078, 'izmir-bayrakli': 1059, 'izmir-bergama': 1092, 'izmir-beydag': 658, 'izmir-bornova': 1096, 'izmir-buca': 1087, 'izmir-cesme': 1097, 'izmir-cigli': 1044, 'izmir-dikili': 1091, 'izmir-foca': 1058, 'izmir-gaziemir': 898, 'izmir-guzelbahce': 835, 'izmir-karabaglar': 1072, 'izmir-karaburun': 1063, 'izmir-karsiyaka': 1057, 'izmir-kemalpasa': 1091, 'izmir-kinik': 889, 'izmir-kiraz': 1069, 'izmir-konak': 1096, 'izmir-menderes': 1097, 'izmir-menemen': 1091, 'izmir-narlidere': 758, 'izmir-odemis': 1096, 'izmir-seferihisar': 1086, 'izmir-selcuk': 853, 'izmir-tire': 1079, 'izmir-torbali': 1096, 'izmir-urla': 1094, 'manisa-ahmetli': 600, 'manisa-akhisar': 1098, 'manisa-alasehir': 1091, 'manisa-demirci': 918, 'manisa-golmarmara': 555, 'manisa-gordes': 1033, 'manisa-kirkagac': 925, 'manisa-koprubasi': 781, 'manisa-kula': 1013, 'manisa-salihli': 1098, 'manisa-sarigol': 1000, 'manisa-saruhanli': 1079, 'manisa-sehzadeler': 1095, 'manis

In [67]:
# 3-) read and preprocess weather.csv

weather = pd.read_csv("./weather.csv")
print(weather.columns) # onemli kolonlar: date, t_apparent:C (hissedilen sicaklik), wind_dir_10m:d (ruzgar yonu),
# wind_speed_10m:ms (ruzgar hizi), prob_precip_1h:p (yagis), ilce

# ilceleri ayir
ilce_weather = {} # keys olarak ilceleri, values olarak o ilcelerin saatlik hava durumklarini tutar
for label, group in weather.groupby("ilce"):
    ilce_weather[label] = group


# tarihleri tarih formatina cevir
#print(ilce_weather["izmir-konak"].dtypes)
for name in ilce_weather.keys():
    tarihler = []
    for date in ilce_weather[name]["date"]:
        tarihler.append(datetime.strptime(date, "%Y-%m-%d %H:%M:%S"))
    ilce_weather[name]["date"] = tarihler
print("bitti")
#print(ilce_weather["izmir-konak"].dtypes)


# tarihleri indexe cevir
for name in ilce_weather.keys():
    ilce_weather[name].set_index("date", inplace=True)


# tarhileri gun olarak birlestir
for name in ilce_weather.keys():
    ilce_weather[name] = ilce_weather[name].resample("D").mean()
print(ilce_weather["izmir-konak"].head())

Index(['date', 'lat', 'lon', 't_2m:C', 'effective_cloud_cover:p',
       'global_rad:W', 'relative_humidity_2m:p', 'wind_dir_10m:d',
       'wind_speed_10m:ms', 'prob_precip_1h:p', 't_apparent:C', 'ilce'],
      dtype='object')
bitti
                lat      lon     t_2m:C  effective_cloud_cover:p  \
date                                                               
2021-01-01  38.4177  27.1283  13.095833                59.033333   
2021-01-02  38.4177  27.1283  13.379167                29.912500   
2021-01-03  38.4177  27.1283  12.587500                69.916667   
2021-01-04  38.4177  27.1283  13.783333                45.604167   
2021-01-05  38.4177  27.1283  13.895833                35.670833   

            global_rad:W  relative_humidity_2m:p  wind_dir_10m:d  \
date                                                               
2021-01-01     65.212500               87.962500      137.558333   
2021-01-02     91.225000               80.720833      134.820833   
2021-01-03     34

C:\Users\ökkeş\AppData\Local\Temp\ipykernel_13168\785132184.py:29: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ilce_weather[name] = ilce_weather[name].resample("D").mean()


In [68]:
# 4-) read and preprocess holidays.csv

holiday = pd.read_csv("./holidays.csv")

print(holiday.head())

holiday["Tarih"] = holiday['Yıl'].astype(str) + '-' + holiday['Ay'].astype(str) + '-' + holiday['Gün'].astype(str)
holiday['Tarih'] = pd.to_datetime(holiday['Tarih'], format='%Y-%m-%d')
holiday.set_index("Tarih", inplace=True)
holiday = holiday.drop(columns=["Yıl", "Ay", "Gün"])

print(holiday.head()) # index olarak tarihi (YY-AA-GG), Bayram_Flag olarak da bayram ismini tutar
print(holiday)

    Yıl  Ay  Gün                              Bayram_Flag
0  2021   1    1                                  Yılbaşı
1  2021   4   23        Ulusal Egemenlik ve Çocuk Bayramı
2  2021   5    1                   Emek ve Dayanışma Günü
3  2021   5   19  Atatürk'ü Anma, Gençlik ve Spor Bayramı
4  2021   7   15           Demokrasi ve Millî Birlik Günü
                                        Bayram_Flag
Tarih                                              
2021-01-01                                  Yılbaşı
2021-04-23        Ulusal Egemenlik ve Çocuk Bayramı
2021-05-01                   Emek ve Dayanışma Günü
2021-05-19  Atatürk'ü Anma, Gençlik ve Spor Bayramı
2021-07-15           Demokrasi ve Millî Birlik Günü
                                                  Bayram_Flag
Tarih                                                        
2021-01-01                                            Yılbaşı
2021-04-23                  Ulusal Egemenlik ve Çocuk Bayramı
2021-05-01                             E

In [71]:
# 5-) merge the train data and holidays, return a new dict called dict_holiday

def merge_holiday(df1, df2):
    merged_df = pd.merge(df1, df2["Bayram_Flag"], left_index=True, right_index=True, how="left")
    #df1["Bayramlar"] = df2["Bayram_Flag"]
    return merged_df

dict_holiday = {}
for name in dict.keys():
    dict_holiday[name] = merge_holiday(dict[name],holiday)
    
print(dict_holiday["izmir-konak"])

                   ilce  bildirimsiz_sum  bildirimli_sum Bayram_Flag
tarih                                                               
2021-01-01  izmir-konak                9               0     Yılbaşı
2021-01-02  izmir-konak                5               0         NaN
2021-01-03  izmir-konak                5               0         NaN
2021-01-04  izmir-konak               11               0         NaN
2021-01-05  izmir-konak               11               0         NaN
...                 ...              ...             ...         ...
2023-12-30  izmir-konak               10               1         NaN
2023-12-31  izmir-konak                9               0         NaN
2024-01-01  izmir-konak               11               0     Yılbaşı
2024-01-02  izmir-konak                9               1         NaN
2024-01-03  izmir-konak                8               1         NaN

[1096 rows x 4 columns]


In [70]:
# 5-) merge the dict_holiday and weather, return merged_all which contains all of the required columns

def merge_weather(df1, df2):
    merged_df = pd.merge(df1, df2, left_index=True, right_index=True, how="left")
    return merged_df

merged_all = {} # key olarak tum ilceler, values olarak kesintiler, bayramlar, hava durumu verilerini tutan df'i tutar
for name in dict_holiday.keys():
    merged_all[name] = merge_weather(dict_holiday[name], ilce_weather[name])
print(merged_all["izmir-konak"].head())

                   ilce  bildirimsiz_sum  bildirimli_sum Bayram_Flag      lat  \
tarih                                                                           
2021-01-01  izmir-konak                9               0     Yılbaşı  38.4177   
2021-01-02  izmir-konak                5               0         NaN  38.4177   
2021-01-03  izmir-konak                5               0         NaN  38.4177   
2021-01-04  izmir-konak               11               0         NaN  38.4177   
2021-01-05  izmir-konak               11               0         NaN  38.4177   

                lon     t_2m:C  effective_cloud_cover:p  global_rad:W  \
tarih                                                                   
2021-01-01  27.1283  13.095833                59.033333     65.212500   
2021-01-02  27.1283  13.379167                29.912500     91.225000   
2021-01-03  27.1283  12.587500                69.916667     34.962500   
2021-01-04  27.1283  13.783333                45.604167     79.4000